In [8]:
import json
import pandas as pd
import numpy as np
from database.operations.chess_com_api import get_profile
import requests
from dotenv import load_dotenv
import os
import psycopg2
from bs4 import BeautifulSoup
import joblib
from database.operations.download_games import download_months
from database.operations.players import read_oponents, read_all_players
from database.database.db_interface import DBInterface
from database.database.models import Player, Game
from database.database.engine import DBSession
from database.operations.models import PlayerCreateData

In [2]:
profile= {'player_name':'pipi'}
profile = PlayerCreateData(**profile)

In [3]:
profile

PlayerCreateData(player_name='pipi', name='None', url='None', title='None', avatar='None', player_id=0, followers=0, country='None', location='None', joined=0, status='None', is_streamer=False, twitch_url='None', verified=False, league='None')

In [22]:
d = joblib.load('games.pkl')

In [23]:
len(d)

TypeError: object of type 'NoneType' has no len()

In [8]:
type(d[0])

database.operations.models.PlayerCreateData

In [9]:
[i for i in d if type(d)==tuple]

[]

In [6]:
load_dotenv()
CONN_STRING = os.getenv("CONN_STRING")

In [7]:
from database.database.engine import init_db

In [8]:
init_db(CONN_STRING)

In [4]:
player_interface = DBInterface(Player)
game_interface = DBInterface(Game)

In [11]:
player_interface.player_exists('mishanick')

True

In [ ]:
player_interface.read_all()

In [60]:
session = DBSession()
white = session.query(
    game_interface.db_class
).where(game_interface.db_class.white =='pat_buchanan').all()
black = session.query(
    game_interface.db_class
).where(game_interface.db_class.black =='pat_buchanan').all()

In [63]:
white[0].black

'hikaru'

In [64]:
black[0].white

'hikaru'

In [65]:
white + black

In [3]:
def get_pgn_item(game, item: str) -> str:
    if item == "Termination":
        return (
            game.split(f"{item}")[1]
            .split("\n")[0]
            .replace('"', "")
            .replace("]", "")
            .lower()
        )
    return (
        game.split(f"{item}")[1]
        .split("\n")[0]
        .replace('"', "")
        .replace("]", "")
        .replace(" ", "")
        .lower()
    )

In [4]:
def clean_games(game: str) -> bool:
    """
    It validates games for length of moves and formatting errors

    """
    game_with_more_than_n_moves = "10. "  # format: 'INT. '

    try:
        game.split("\n\n")[1][:-4]
    except:
        return False
    if not game_with_more_than_n_moves in game.split("\n\n")[1][:-4]:
        return False
    if game.split("\n\n")[1][:-4].startswith("1... "):
        return False
    if (
        "/"
        in game.split(f"TimeControl")[1]
        .split("\n")[0]
        .replace('"', "")
        .replace("]", "")
        .replace(" ", "")
        .lower()
    ):
        return False
    return True

In [5]:
valid_dates = [(2023, 1)]

In [6]:
player_name = 'hikaru'

In [9]:
w = download_months(player_name, valid_dates)

GOT 517 games
downloading over
returned months = 517


In [12]:
w[0]

'[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2023.01.27"]\n[Round "-"]\n[White "DanielNaroditsky"]\n[Black "Hikaru"]\n[Result "0-1"]\n[CurrentPosition "8/8/8/3k4/p1q5/8/3K4/8 w - -"]\n[Timezone "UTC"]\n[ECO "B12"]\n[ECOUrl "https://www.chess.com/openings/Caro-Kann-Defense-Advance-Short-Variation-with-4-Nf3...5.Be2-Ne7-6.O-O-h6-7.Nbd2"]\n[UTCDate "2023.01.27"]\n[UTCTime "07:16:52"]\n[WhiteElo "3283"]\n[BlackElo "3317"]\n[TimeControl "60"]\n[Termination "Hikaru won by resignation"]\n[StartTime "07:16:52"]\n[EndDate "2023.01.27"]\n[EndTime "07:19:27"]\n[Link "https://www.chess.com/game/live/68542929653"]\n\n1. e4 {[%clk 0:01:00]} 1... c6 {[%clk 0:01:00]} 2. d4 {[%clk 0:00:59.5]} 2... d5 {[%clk 0:00:59.9]} 3. e5 {[%clk 0:00:59.1]} 3... Bf5 {[%clk 0:00:59.5]} 4. Nf3 {[%clk 0:00:59]} 4... e6 {[%clk 0:00:59.1]} 5. Be2 {[%clk 0:00:58.9]} 5... Ne7 {[%clk 0:00:58.5]} 6. O-O {[%clk 0:00:58.8]} 6... h6 {[%clk 0:00:58.2]} 7. Nbd2 {[%clk 0:00:58.3]} 7... Nd7 {[%clk 0:00:57.9]} 8. Ne1 {[%clk 0:0

In [14]:
get_pgn_item(w[0], "Link")

'https://www.chess.com/game/live/68542929653'

In [36]:
games = [game for game in w if clean_games(game)]

In [37]:
len(w)

1344

In [15]:
int(get_pgn_item(w[0], "[Link").split("/")[-1])

68542929653

In [41]:
players_and_game_id = np.array(
        [
            [
                get_pgn_item(game, "White").lower(),
                get_pgn_item(game, "Black").lower(),
                int(get_pgn_item(game, "[Link").split("/")[-1]),
            ]
            for game in games
        ]
    )

In [42]:
players_and_game_id

array([['hikaru', 'njal28', '74057526727'],
       ['njal28', 'hikaru', '74057452511'],
       ['hikaru', 'njal28', '74056933645'],
       ...,
       ['hikaru', 'gmleonlivaic', '69010411945'],
       ['hikaru', 'oleksandr_bortnyk', '69010258731'],
       ['hikaru', 'danielnaroditsky', '69009850063']], dtype='<U25')

In [3]:
def create_month(params: list) -> None:
    """
    Download a month, splits the games and return a list of pgn games.
    """
    pgn = download_month(params["player_name"], params["year"], params["month"])
    if pgn == False:
        params["return_games"].extend(False)
        return
    games = pgn.read().split("\n\n\n")
    return games

In [4]:
pgn = download_month('hikaru', '2023', '1')

In [6]:
pgn = pgn.read().split("\n\n\n")

In [2]:
def get_ask_connection():
    CONN_STRING = os.getenv("PSYCOPG2_CONN_STRING")
    return psycopg2.connect(CONN_STRING, port = 5433)


def player_exists_at_db(player_name: str):
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            f"select player_name from player where player_name='{player_name}'"
        )
        result = curs.fetchall()
    if len(result) == 1:
        return True
    return False
def get_games(player_name: str):
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            f"select id from game where white='{player_name}'"
        )
        result = curs.fetchall()
    return result
def get_all_players():
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            f"select player_name from player"
        )
        result = curs.fetchall()
    return result
def open_request(sql_question:str):
    conn = get_ask_connection()
    with conn.cursor() as curs:
        curs.execute(
            sql_question
        )
        result = curs.fetchall()
    return result

In [4]:
open_request(f"select id from game where game.id = 132")

[]

In [ ]:
get_all_players()

In [6]:
player_exists_at_db('hikaru'.lower())

True

In [3]:
get_profile('sdssssdfssssssssssssss')

False

In [5]:
player_name = 'Pat_Buchanan'

In [6]:
prof = get_profile(player_name)

In [7]:
prof

{'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/29584920.b3462b3c.200x200o.b4b455f59035.jpeg',
 'player_id': 29584920,
 '@id': 'https://api.chess.com/pub/player/pat_buchanan',
 'url': 'https://www.chess.com/member/Pat_Buchanan',
 'name': 'Pat Buchanan',
 'username': 'pat_buchanan',
 'followers': 98,
 'country': 'https://api.chess.com/pub/country/MX',
 'location': 'Ciudad de M&eacute;xico',
 'last_online': 1697477530,
 'joined': 1471850943,
 'status': 'basic',
 'is_streamer': False,
 'verified': False,
 'league': 'Legend'}

In [5]:
type(prof)

dict

In [8]:
country = prof.pop('country').split('/')[-1]

In [9]:
country

'MX'

In [3]:
member_exists(player_name = 'hikaru')

True

In [8]:
from database.operations.models import PlayerCreateData, PlayerResult

In [17]:
profile = get_profile('pat_buchanan')

In [18]:
profile.pop('@id')

'https://api.chess.com/pub/player/pat_buchanan'

In [19]:
p_name = profile.pop('name')

In [20]:
p_name

'Pat Buchanan'

In [11]:
profile

{'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/29584920.b3462b3c.200x200o.b4b455f59035.jpeg',
 'player_id': 29584920,
 'url': 'https://www.chess.com/member/Pat_Buchanan',
 'name': 'Pat Buchanan',
 'username': 'pat_buchanan',
 'followers': 98,
 'country': 'https://api.chess.com/pub/country/MX',
 'location': 'Ciudad de M&eacute;xico',
 'last_online': 1697477530,
 'joined': 1471850943,
 'status': 'basic',
 'is_streamer': False,
 'verified': False,
 'league': 'Legend'}

In [13]:
a = PlayerCreateData(**profile)

In [16]:
a.model_dump()

{'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/29584920.b3462b3c.200x200o.b4b455f59035.jpeg',
 'player_id': 29584920,
 'url': 'https://www.chess.com/member/Pat_Buchanan',
 'name': 'Pat Buchanan',
 'username': 'pat_buchanan',
 'followers': 98,
 'country': 'https://api.chess.com/pub/country/MX',
 'location': 'Ciudad de M&eacute;xico',
 'last_online': 1697477530,
 'joined': 1471850943,
 'status': 'basic',
 'is_streamer': False,
 'verified': False,
 'league': 'Legend'}

SyntaxError: invalid syntax (<ipython-input-25-e62177203b71>, line 1)

In [ ]:
class Person(db.Model):
    id = db.Column(db.Integer, primary_key=True)

    user = db.relationship("BankSlip", foreign_keys='BankSlip.person_user_id', back_populates="person_user")
    reference = db.relationship("BankSlip", foreign_keys='BankSlip.person_ref_id', back_populates="person_reference")